## Save the products df as "products" and query df as "queries" 

In [2]:
import pandas as pd
import numpy as np
queries = {'query_id': [0, 0], 
     'query': ['!awnmower tires without rims	', 'Bawnmower tires without rims	'],
     'query_locale': ['us','us'], 
     'product_id': ['B0188A3QRM', 'B075VXJ9VG'],
     'esci_label': ['Exact', 'Sub']}

queries = pd.DataFrame(data=queries)


d = {'product_id': ['B0188A3QRM', 'B075VXJ9VG'], 
     'product_title': ['Amazon Basics Woodcased #2 Pencils, Unsharpened', 'BAZIC Pencil #2 HB Pencils, Latex Free Eraser'],
     'product_description': [np.NaN, '<p><strong>BACK TO BAZIC</strong></p><p>Our go'], 
     'product_bullet_point': ['144 woodcase #2 HB pencils made from high-quality', '&#11088; UN-SHARPENED #2 PREMIUM PENCILS.'],
     'product_brand': ['Amazon Basics	', 'BAZIC Products	'],
     'product_color_name': [ 'Yellow', '12-count'],
     'product_locale': [ 'us', 'us']}
products = pd.DataFrame(data=d)

In [3]:
! pip install transformers


     |████████████████████████████████| 4.0 MB 18.6 MB/s 
     |████████████████████████████████| 6.6 MB 37.0 MB/s 
     |████████████████████████████████| 77 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 37.2 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Importing modules
import pandas as pd
import numpy as np

from transformers import AutoModel
from transformers import AutoTokenizer
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


# Merge the two DFs
df = pd.merge(products, queries,
    how="left",
    on= "product_id")
df


## STEP 1
# First create column for concatting all description columns for products
df['Combined'] = df['query'].astype(str) + ', ' + df['product_title'].astype(str) + ', ' + df['product_description'].astype(str) + ', ' + df['product_bullet_point'].astype(str)  + ', ' + df['product_brand'].astype(str) + ', ' + df['product_color_name'].astype(str) + ', ' + df['product_locale'].astype(str)
df = pd.DataFrame(data=df)

## STEP 2
# Then break up each entry in the "Combined" column into sets of 512 characters
df['Split_String'] = ""
broken_text = []
s = df['Combined']
for text in s:
  n = 512
  a = [text[i:i+n] for i in range(0, len(text), n)]
  broken_text.append(a)

## STEP 3
# Then i put it into another column "Split_String"(idk why) but in list form, so i can loop thru it
X = df["Split_String"]
i = 0
while i < len(X): #change 10 to length of dataframe
  X[i] = broken_text[i]
  i+=1

## STEP 4: 
# calculating bert numbers for each set of 512 sets
df['BERT Numbers'] = ""
X = df["Split_String"]
Bert = df['BERT Numbers']

i=0
X = df["Split_String"]
while i < len(X):
  embeddings=[]
  for o in X[i]:
    text = o
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
      outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state)
  Bert[i]= embeddings  
  i+=1


## STEP 5:
# getting the averages and storing them in "Mean of Bert"
y = df['BERT Numbers']
df['Mean of Bert'] = ""

Means = df['Mean of Bert']
i=0
Averages = []
while i < len(y): # num of rows in data set (y is the column of bert nums)
  concated_means = []
  j=0
  while j < len(y[i]): # num of bert numbers per product. (sets of 512)
    concated_means.append(torch.mean(y[i][j], dim=1))
    j+=1
  y_concats = torch.cat(concated_means)
  y_avg = torch.mean(y_concats, dim = 0)
  Means[i]= y_avg
  i+=1






In [7]:
df

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,query_id,query,query_locale,esci_label,Combined,Split_String,BERT Numbers,Mean of Bert
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened",NaN,144 woodcase #2 HB pencils made from high-quality,Amazon Basics\t,Yellow,us,0,!awnmower tires without rims\t,us,Exact,"!awnmower tires without rims\t, Amazon Basics ...","[!awnmower tires without rims\t, Amazon Basics...","[[[tensor([-1.8273e-01, -3.5165e-01, 1.6223e-...","[tensor(-0.0632), tensor(0.0138), tensor(0.227..."
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser",<p><strong>BACK TO BAZIC</strong></p><p>Our go,&#11088; UN-SHARPENED #2 PREMIUM PENCILS.,BAZIC Products\t,12-count,us,0,Bawnmower tires without rims\t,us,Sub,"Bawnmower tires without rims\t, BAZIC Pencil #...","[Bawnmower tires without rims\t, BAZIC Pencil ...","[[[tensor([-2.0032e-01, -1.5358e-01, -1.2254e-...","[tensor(-0.0205), tensor(0.0058), tensor(0.357..."
